# Pipe for create embeddings

## Imports

In [51]:
import torch
from transformers import AutoTokenizer, AutoModel
import functions
import json

import pandas as pd
import numpy as np
import seaborn as sns
import math

import sklearn.metrics as metrics

## Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

## Data

In [3]:
df = pd.read_json("CLR10.jsonl", lines=True)
big_boys = df.copy()

In [4]:
big_boys["completion"] = big_boys["completion"].str.len()
big_boys = big_boys[big_boys.completion.astype(int) > 1000]

In [5]:
df.shape

(1640, 2)

## Get Embeddings

In [6]:
data = []
with open('CLR10.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [33]:
data_1 = []
with open('CLR1.jsonl_results.jsonl') as f:
    for line in f:
        data_1.append(json.loads(line))

In [36]:
data_1[0]["passed"]

False

In [37]:
def get_passed(data):
    passes = []
    for i in data:
        passes.append(i["passed"])
    return passes

In [7]:
def split_into_blocks(samples, block_size):
    return [samples[i:i+block_size] for i in range(0, len(samples), block_size)]

In [8]:
blocks = split_into_blocks(data, 10)
blocks_0 = blocks[0]

In [14]:
emb = []
j = 0
for b in blocks:
    res = functions.form_emb(b, model, tokenizer)
    emb.append(res)
    print(j)
    j += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


In [25]:
len(emb)

164

In [26]:
def get_entropie(embeddings):
    entropies = []
    for x in embeddings:
        entropy = functions.kernel_entropy(x, kernel=lambda x, y: metrics.pairwise.rbf_kernel(x, y, gamma=None))
        entropies.append(entropy)
    return entropies

## Pipe

In [55]:
kernel_entropy_values = get_entropie(emb)
kernel_entropy_values_flipped = []
for i in kernel_entropy_values:
    kernel_entropy_values_flipped.append(abs(i))
    

In [56]:
kernel_entropy_values_flipped

[0.9906867133246527,
 0.9682647705078125,
 0.9707090589735243,
 0.9903168572319878,
 0.9212972852918837,
 0.9754938761393229,
 0.869362301296658,
 0.9773200141059027,
 0.9749673631456163,
 0.9839932759602864,
 0.9735575358072917,
 0.9128739251030816,
 0.9762363857693143,
 0.9879859076605902,
 0.9941034952799479,
 0.9768653869628906,
 0.9672409057617187,
 0.9234320746527778,
 0.9815115186903212,
 0.9100140889485677,
 0.9689888848198784,
 0.9166670905219184,
 0.9953699747721354,
 0.9807843526204427,
 0.9145840115017361,
 0.9767080518934462,
 0.9756429884168837,
 0.9758702596028646,
 0.978177981906467,
 0.9874637179904514,
 0.9849557664659289,
 0.9841891818576389,
 0.9680437723795573,
 0.9768463982476129,
 0.9884674919976129,
 0.9680828518337674,
 0.9915179782443576,
 0.9796989440917969,
 0.98987791273329,
 0.9239468044704862,
 0.9937407599555121,
 0.9645234849717882,
 0.9710443284776475,
 0.9281242370605469,
 0.9771308898925781,
 0.9937500847710503,
 0.9179361131456163,
 0.97551498413085

In [38]:
is_prediction_correct = get_passed(data_1)

In [58]:
kent_auroc = metrics.roc_auc_score(
    is_prediction_correct,
    kernel_entropy_values_flipped
)

In [59]:
kent_auroc

0.5728451563691839

In [47]:
def compare(x, y):
    compared = []
    for i in range(0, 164):
        res = (x[i], y[i])
        compared.append(res)
    return compared

In [57]:
res = compare(kernel_entropy_values_flipped, is_prediction_correct)

In [49]:
print(len(res))

164


In [50]:
for t in res:
    print(t)

(-0.9906867133246527, False)
(-0.9682647705078125, False)
(-0.9707090589735243, True)
(-0.9903168572319878, True)
(-0.9212972852918837, True)
(-0.9754938761393229, True)
(-0.869362301296658, False)
(-0.9773200141059027, True)
(-0.9749673631456163, True)
(-0.9839932759602864, True)
(-0.9735575358072917, False)
(-0.9128739251030816, False)
(-0.9762363857693143, True)
(-0.9879859076605902, True)
(-0.9941034952799479, True)
(-0.9768653869628906, True)
(-0.9672409057617187, True)
(-0.9234320746527778, False)
(-0.9815115186903212, True)
(-0.9100140889485677, False)
(-0.9689888848198784, False)
(-0.9166670905219184, True)
(-0.9953699747721354, True)
(-0.9807843526204427, True)
(-0.9145840115017361, True)
(-0.9767080518934462, True)
(-0.9756429884168837, False)
(-0.9758702596028646, True)
(-0.978177981906467, True)
(-0.9874637179904514, True)
(-0.9849557664659289, True)
(-0.9841891818576389, True)
(-0.9680437723795573, False)
(-0.9768463982476129, False)
(-0.9884674919976129, True)
(-0.9680828

In [11]:
sample = df.loc[739]["completion"]

print(len(sample))

1792


In [12]:
res = functions.get_embedding(model, tokenizer, sample)

In [13]:
res.shape

torch.Size([512, 768])

In [19]:
big_boys

,task_id,completion
204,HumanEval/20,1297
218,HumanEval/21,1027
245,HumanEval/24,1154
247,HumanEval/24,1319
262,HumanEval/26,1896
503,HumanEval/50,1260
545,HumanEval/54,1280
657,HumanEval/65,1224
673,HumanEval/67,1024
678,HumanEval/67,1389


## Debug

In [5]:
data = []
with open('CLR10.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [7]:
data[10]["task_id"]

'HumanEval/1'

In [80]:
print(len(block_0))

10


In [109]:
def form_emb(data):
    embeddings = []
    for line in data:
        print(line["task_id"])
        try:
            embedding = functions.get_embedding(model, tokenizer, line["completion"])
            fix = embedding.detach()
            embeddings.append(fix)
        except:
            continue
            
    return embeddings

In [110]:
block_emb = form_emb(block_0)

HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2


In [ ]:
sample0_emb = functions.get_embedding(model, tokenizer, sample0)
sample1_emb = functions.get_embedding(model, tokenizer, sample1)
sample2_emb = functions.get_embedding(model, tokenizer, sample2)
sample3_emb = functions.get_embedding(model, tokenizer, sample3)
sample4_emb = functions.get_embedding(model, tokenizer, sample4)
sample5_emb = functions.get_embedding(model, tokenizer, sample5)
sample6_emb = functions.get_embedding(model, tokenizer, sample6)
sample7_emb = functions.get_embedding(model, tokenizer, sample7)
sample8_emb = functions.get_embedding(model, tokenizer, sample8)
sample9_emb = functions.get_embedding(model, tokenizer, sample9)


embedding_room_detach = [sample0_emb[0].detach(), sample1_emb[0].detach(), sample2_emb[0].detach(), sample3_emb[0].detach(), sample4_emb[0].detach(), sample5_emb[0].detach(), sample6_emb[0].detach(), sample7_emb[0].detach(), sample8_emb[0].detach(), sample9_emb[0].detach()]
embedding_room_0 = [sample0_emb[0], sample1_emb[0], sample2_emb[0], sample3_emb[0], sample4_emb[0], sample5_emb[0], sample6_emb[0], sample7_emb[0], sample8_emb[0], sample9_emb[0]]
embedding_room_1 = [sample0_emb, sample1_emb, sample2_emb, sample3_emb, sample4_emb, sample5_emb, sample6_emb, sample7_emb, sample8_emb, sample9_emb]



embeddings = [i.mean(0).detach().numpy() for i in embedding_room_1]
embeddings = np.array(embeddings)

embeddings_nomean = [i.mean(0).detach().numpy() for i in embedding_room_1]
embeddings_nomean = np.array(embeddings)